In [ ]:
import os
from glob import glob
import json
from tqdm import tqdm
def create_dir(base,ext):
    '''
        creates a directory extending base
        args:
            base    =   base path 
            ext     =   the folder to create
    '''
    _path=os.path.join(base,ext)
    if not os.path.exists(_path):
        os.mkdir(_path)
    return _path

bw_path="/backup/RAW/bangla_writing/raw/raw/"
save_path="/backup/RAW/DET/"
save_path=create_dir(save_path,"bw")
img_dir=create_dir(save_path,"images")
gt_dir=create_dir(save_path,"gts")

img_paths=[img_path for img_path in tqdm(glob(os.path.join(bw_path,"*.jpg")))]


In [ ]:

import numpy as np
import shutil
def save_data(img_path):
    labels=[]
    # json_path
    json_path=img_path.replace("jpg","json")
    # label
    label_json = json.load(open(json_path,'r'))
    box_text_list = []
    # get word idx
    for idx in range(len(label_json['shapes'])):
        # label
        label=str(label_json['shapes'][idx]['label'])
        # crop bbox
        xy=label_json['shapes'][idx]['points']
        # crop points
        x1 = int(np.round(xy[0][0]))
        y1 = int(np.round(xy[0][1]))
        x2 = int(np.round(xy[1][0]))
        y2 = int(np.round(xy[1][1]))
        box=[x1,y1,x2,y1,x2,y2,x1,y2]
        if label=="*":
            text = "###"
        else:
            text=label
        
        box_text_list.append(','.join(map(str, box)))
        box_text_list.append(',')
        box_text_list.append(text)
        box_text_list.append('\n')
    
    # 
    base_=os.path.basename(img_path).split(".")[0]
    
    img_dst=os.path.join(img_dir,f"{base_}.jpg")
    shutil.copy(img_path,img_dst)
    
    gts_text_file=os.path.join(gt_dir,f"{base_}.txt")
    with  open(gts_text_file, "w") as file:
        makeitastring = ''.join(map(str, box_text_list))
        file.write(makeitastring)
        file.close()


In [ ]:
for img_path in tqdm(img_paths):
    save_data(img_path)